### TF 基础和运算

TF 的基本知识的笔记，灵感来自于 [Stanford CS20si 课程](http://web.stanford.edu/class/cs20si/syllabus.html)，enjoy！

1.什么是 tensor?
- ndarray

2.TF 将图的定义和执行分开
- 组装图
- 使用 Session 来执行

3.TF graph 的组成
- nodes: operators, variables, constants
- edges: tensors

4.Session 的作用是什么？
- 封装运行时环境，使得 op 被执行，tensor 的值被计算
- 为当前的变量值分配内存

5.为什么使用图？
- 节省计算，计算时只需要关注变量所依赖的子图
- 将计算拆分为小的、更容易微分的部分，这样 TF 就容易计算每个节点的导数
- 分布式计算，将互不依存的图分配到不同的计算节点上
- 许多机器学习模型都可以使用计算图表示

In [ ]:
# 将部分的运算分布到不同的计算节点
# Creates a graph.
with tf.device('/gpu:2'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name='b')
  c = tf.multiply(a, b)

# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# Runs the op.
print(sess.run(c))
sess.close()

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # 设置后，程序执行时就不会出现 warning
import tensorflow as tf

In [16]:
# 最简单的 TF 程序
a = tf.constant(2, name='a')
b = tf.constant(3, name='b')
x = tf.add(a, b, name='add')
with tf.Session() as sess:
    print(sess.run(x))

In [15]:
# 使用 Tensorboard 来查看计算图。
# summary writer 在图定义好之后、session 运行前创建，这样才会记录所有的图内容
# 使用命令 tensorboard --logdir='./graphs' --port 6006 来运行 tensorboard
# 在浏览器中查看

a = tf.constant(2, name='a')
b = tf.constant(3, name='b')
x = tf.add(a, b, name='add')

writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
writer.close()

with tf.Session() as sess:
    print(sess.run(x))

In [ ]:
# 常量 constant 操作

# 定义常量
const = tf.constant([[0, 1], [2, 3]], name='const')
# 全零常量，类似于 numpy.zeros
zeros = tf.zeros([2, 3], tf.float32, name='zeros')
# 全零常量，类似于 numpy.zeros_like
zeros_like = tf.zeros_like(zeros, dtype=tf.int32, name='zeros_like')
# 全 1 常量，类似于 numpy.ones
ones = tf.ones([2, 3], tf.float32, name='ones')
# 全 1 常量，类似于 numpy.ones_like
ones_like = tf.ones_like(ones, dtype=tf.int32, name='ones_like')
# 填充常量，类似于 numpy.full
fill = tf.fill([2, 3], 8, name='fill') # [[8, 8, 8], [8, 8, 8]]
# 定义常量序列
lin_space = tf.lin_space(10.0, 14.0, 10, name='linspace')
rang = tf.range(2, 18, 3, name='range') # 注意，和 np 不一样的是，这里的 rang 不能迭代，for _ in tf.range(4): # TypeError
# 给常量赋随机值
tf.random_normal
tf.truncated_normal
tf.random_uniform
tf.random_shuffle
tf.random_crop
tf.random_gamma
# 设置随机种子
tf.set_random_seed(22)

In [10]:
# 数学操作

# tensorflow 里面有非常多的除法操作，但都大同小异，faint！
a = tf.constant([2, 2], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')
with tf.Session() as sess:
    print(sess.run(tf.div(b, a)))             # ⇒ [[0 0] [1 1]]
    print(sess.run(tf.divide(b, a)))          # ⇒ [[0. 0.5] [1. 1.5]]
    print(sess.run(tf.truediv(b, a)))         # ⇒ [[0. 0.5] [1. 1.5]]
    print(sess.run(tf.floordiv(b, a)))        # ⇒ [[0 0] [1 1]]
    print(sess.run(tf.realdiv(b, a)))         # ⇒ # Error: only works for real values
    print(sess.run(tf.truncatediv(b, a)))     # ⇒ [[0 0] [1 1]]
    print(sess.run(tf.floor_div(b, a)))       # ⇒ [[0 0] [1 1]]

# 多个 tensor 相加
a = tf.constant([10, 20], name='a')
b = tf.constant([2, 3], name='b')
tf.add_n([a, b, b]) # ⇒ a + b + b

# 两种乘法操作
tf.multiply(a, b) # ⇒ [20 60] element-wise
tf.tensordot(a, b, 1) # ⇒ 80，这里的 1 表示矩阵相乘

In [14]:
# tf 的数据类型
# tf 的数据类型和 np 的数据类型无缝衔接
tf.int32 == np.int32 # true
tf.ones([2, 2], np.float32) # [[1.0 1.0], [1.0 1.0]]
tf.Session.run(fetches) # 如果 fetch 是个 tensor，则输出是 numpy ndarray

In [18]:
# tf 中的变量
# 常量有啥问题？
# 1.常量不能变化，weights 和 bias 需要不断地更新，所以不能用常量。
# 2.常量的值是存在 graph 中的，当常量的数量非常多，例如 weight 有几百万个值，加载 graph 就会非常缓慢。变量是存在 parameter server 中的。

# graph 是用 protobuf 格式存储的
# protobuf = protocol buffer, Google's language-neutral, platform-neutral, extensible mechanism for serializing structured data – think XML, but smaller, faster, and simpler.
# 打印出 graph 的 protobuf：
print(tf.get_default_graph().as_graph_def())

# 设置变量的值
# 不推荐的方法
s = tf.Variable(2, name="scalar")
m = tf.Variable([[0, 1], [2, 3]], name="matrix")
W = tf.Variable(tf.zeros([784,10]))
# 推荐的方法，使用 tf.get_variable 来定义变量
s = tf.get_variable("scalar", initializer=tf.constant(2))
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("big_matrix", shape=(784, 10), initializer=tf.zeros_initializer())
# 为什么 tf.Variable 首字母是大写，tf.constant 是小写
# - 因为 tf.constant 是操作，tf.Variable 是类

# 变量的一些操作
x.initializer
x.value() # 读操作
x.assign(...) # 赋值操作

# 在 sess 执行阶段，变量需要初始化，否则会报错
# FailedPreconditionError: Attempting to use uninitialized value Variable
# 初始化最简单的方式
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # initializer 是一个 op，需要在 sess 中才能执行
    # 或者列出需要初始化的变量
    sess.run(tf.variables_initializer([s, m]))
    # 或者单独地初始化
    sess.run(s.initializer)

# 查看变量的值，使用 eval
W = tf.Variable(tf.truncated_normal([700, 10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W) # <tf.Variable 'Variable:0' shape=(700, 10) dtype=float32_ref>
    print(W.eval()) # [[ 7.5906491e-01 -6.2605661e-01  1.8686393e-02 ...
    print(sess.run(W)) # 同上
    
# 给变量赋值，使用 assign
W = tf.Variable(10)
assign_op = W.assign(100) # 注意，这里定义了一个 op，需要在 sess 中执行，才能对 W 赋值。如果不执行该 op，将不会生效。
assign_times_two = W.assign(2 * W)
assign_add = W.assign_add(10)
assign_sub = W.assign_sub(2)
with tf.Session() as sess:
    sess.run(W.initializer)
    sess.run(assign_op)
    print(W.eval()) # >> 100
    sess.run(assign_times_two) # 多次操作
    print(W.eval()) # >> 200
    sess.run(assign_times_two)
    print(W.eval()) # >> 400

# 每个 sess 维持自己的变量拷贝，互不干扰
W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()
sess1.run(W.initializer)
sess2.run(W.initializer)
print(sess1.run(W.assign_add(10))) # >> 20
print(sess2.run(W.assign_sub(2))) # >> 8
sess1.close()
sess2.close()

In [ ]:
# 控制依赖关系：指定 d,e 必须在 a,b,c 之后执行
with g.control_dependencies([a, b, c]):
  d = ...
  e = ...

In [ ]:
# Placeholder 占位符
# 定义 f(x, y) = 2x + y
# 定义图的时候，并不知道具体的值是什么，就用占位符来定义

# 定义一个长度为 3 的占位符
a = tf.placeholder(tf.float32, shape=[3]) # shape 默认为 None，表示任何 shape 都可以接受。调试的时候就悲剧了。并且打乱了所有的形状推断，导致许多 op 出错。
b = tf.constant([5,5,5], tf.float32)
c = a + b

with tf.Session() as sess:
    print(sess.run(c)) # InvalidArgumentError: a doesn’t an actual value
    # 使用字典来提供占位符的值
    print(sess.run(c, feed_dict={a: [1,2,3]})) # [6,7,8]

# 不只是 placeholder，可以通过 dict 喂任何可以喂的 tensor
# 在调试的时候，这个方法非常有用
a = tf.add(2, 5)
b = tf.multiply(a, 3)
with tf.Session() as sess:
    sess.run(b, feed_dict={a: 15}) # 将 a = 15 喂入 op 中。

In [40]:
# 延迟加载带来的问题

# 1 普通加载的例子
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y) # 执行图中的操作前，先定义好这个 op

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(z)

# 2 延迟加载的例子
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(tf.add(x, y)) # 这样就可以省一行代码，聪明！

# 普通加载中，add 操作被添加到图中 1 次
# 延迟加载中，add 操作被添加到图中 10 次
# 如果成千上万次，图就会爆掉，加载会非常缓慢，这是很常见的错误

In [ ]:
# TF 的流程控制
# TF 的图是在执行前定义好的，因此需要将每一个可能的子图定义好
tf.group, tf.count_up_to, tf.cond, tf.case, tf.while_loop # 流程控制
tf.equal, tf.not_equal, tf.less, tf.greater, tf.where # 比较操作
tf.logical_and, tf.logical_not, tf.logical_or, tf.logical_xor # 逻辑位操作
tf.is_finite, tf.is_inf, tf.is_nan, tf.Assert, tf.Print # 调试操作